Cài đặt Spark

In [2]:
#!wget -q https://archive.apache.org/dist/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
#!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [ ]:
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
#os.environ["SPARK_HOME"] = "/content/spark-3.5.2-bin-hadoop3"

In [3]:
#import findspark
#findspark.init()
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("local[*]").getOrCreate()
#spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
#spark.conf.set("spark.sql.shuffle.partitions", 1800)
#spark

from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("DataIntegration") \
    .getOrCreate()

Chuẩn bị mô hình phân nhóm các ngành công nghiệp


In [ ]:
#!pip uninstall -qf torch torchvision torchaudio
#!pip install -qf torch torchvision torchaudio

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("sampathkethineedi/industry-classification")
model = AutoModelForSequenceClassification.from_pretrained("sampathkethineedi/industry-classification")

industry_tags = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

Chuẩn bị dữ liệu

https://www.projectpro.io/recipes/read-data-from-hdfs-pyspark

In [4]:
!cp '/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/Jobs_NYC_Postings.7z' .
!cp '/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/large_naukri_data.7z' .

In [5]:
!p7zip -d  large_naukri_data.7z 1> /dev/null
!p7zip -d  Jobs_NYC_Postings.7z 1> /dev/null

Xử lý dữ liệu từ ấn độ

In [6]:
#indian_jobs=spark.read.json("hdfs://localhost:9000/user/u1/large_naukri_data.json")
indian_jobs = spark.read.json('large_naukri_data.json')
old = indian_jobs.schema.names
new = [item.replace(' ', '_') for item in old]
indian_jobs = indian_jobs.toDF(*new)
indian_jobs.createOrReplaceTempView('indian_view')

In [7]:
indian_jobs.show(5)

+---------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+------------+--------+--------------------+--------------+----------+--------------+----------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+------+----------+------+
|_corrupt_record|                 _id|        company_name|          crawled_at|currency|         description|            industry|      job_id|job_type|           locations|max_experience|max_salary|min_experience|min_salary|                  pg|          posted_at|                 ppg|                role|       salary_label|              skills|               title|                  ug|                 url|vacany|view_count|walkin|
+---------------+--------------------+--------------------+--------------------+--------+--------------------+------------

In [ ]:
indian_jobs.schema.names

['_corrupt_record',
 '_id',
 'company_name',
 'crawled_at',
 'currency',
 'description',
 'industry',
 'job_id',
 'job_type',
 'locations',
 'max_experience',
 'max_salary',
 'min_experience',
 'min_salary',
 'pg',
 'posted_at',
 'ppg',
 'role',
 'salary_label',
 'skills',
 'title',
 'ug',
 'url',
 'vacany',
 'view_count',
 'walkin']

In [ ]:
from pyspark.sql.functions import when
from pyspark.sql.functions import lit, udf
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType


def id_industry_tags_udf(industry, role, skills):
    return industry_tags(f"{industry}, {role}, {skills}")[0]['label']

def id_industry_tags_udf_score(industry, role, skills):
    return industry_tags(f"{industry}, {role}, {skills}")[0]['score']

# Register the UDF
id_industry_tags_udf = udf(id_industry_tags_udf, StringType())
id_industry_tags_udf_score = udf(id_industry_tags_udf_score, DoubleType())

id_col = ['industry', #'job_type', #'min_experience','max_experience',
          'min_salary', 'max_salary', 'skills', 'vacany', 'role']
id_df = spark.sql(f'select {",".join(id_col)} from indian_view where industry is not null').collect()
id_df = spark.createDataFrame(id_df)

id_df = id_df.withColumn('max_salary', id_df['max_salary'] * 0.012)
id_df = id_df.withColumn('min_salary', id_df['min_salary'] * 0.012)
#id_df = id_df.withColumn('job_type', when(id_df.job_type == 'fulltime', 'F').otherwise('P'))

id_df = id_df.withColumn('description', id_industry_tags_udf(id_df['industry'], id_df['role'], id_df['skills']))
#id_df = id_df.withColumn('score', id_industry_tags_udf_score(id_df['industry'], id_df['role'], id_df['skills']))

id_df = id_df.drop('industry', 'role', 'skills')

id_df.show(5)

+----------+----------+------+--------------------+
|min_salary|max_salary|vacany|         description|
+----------+----------+------+--------------------+
|    6000.0|   10800.0|     1|IT Consulting & O...|
|     600.0|     600.0|     0|Construction & En...|
|    1800.0|    2400.0|     5|Health Care Techn...|
|    9600.0|   15600.0|     1|IT Consulting & O...|
|    3600.0|    4500.0|   200|Data Processing &...|
+----------+----------+------+--------------------+
only showing top 5 rows



xử lý dữ liệu từ nữu ước

In [8]:
nyc_jobs = spark.read.csv('Jobs_NYC_Postings.csv', header=True)
old = nyc_jobs.schema.names
new = [item.replace(' ', '_') for item in old]
new = [item.replace('#', '_') for item in new]
new = [item.replace('/', '_') for item in new]
new = [item.replace('-', '_') for item in new]
nyc_jobs = nyc_jobs.toDF(*new)
nyc_jobs.createOrReplaceTempView('nyc_view')

In [9]:
nyc_jobs.show(5)

+------+--------------------+------------+--------------+--------------------+--------------------+--------------------+-------------+-----+--------------------+-----------------------------+--------------------+-----------------+---------------+----------------+--------------------+--------------------+--------------------+-------------------------+--------------------+----------------------+--------------------+-----------+------------------+-------------------+---------------------+------------+-----------+---------------+------------+
|Job_ID|              Agency|Posting_Type|__Of_Positions|      Business_Title| Civil_Service_Title|Title_Classification|Title_Code_No|Level|        Job_Category|Full_Time_Part_Time_indicator|        Career_Level|Salary_Range_From|Salary_Range_To|Salary_Frequency|       Work_Location|  Division_Work_Unit|     Job_Description|Minimum_Qual_Requirements|    Preferred_Skills|Additional_Information|            To_Apply|Hours_Shift|   Work_Location_1|Recruit

In [ ]:
nyc_jobs.schema.names

['Job_ID',
 'Agency',
 'Posting_Type',
 '__Of_Positions',
 'Business_Title',
 'Civil_Service_Title',
 'Title_Classification',
 'Title_Code_No',
 'Level',
 'Job_Category',
 'Full_Time_Part_Time_indicator',
 'Career_Level',
 'Salary_Range_From',
 'Salary_Range_To',
 'Salary_Frequency',
 'Work_Location',
 'Division_Work_Unit',
 'Job_Description',
 'Minimum_Qual_Requirements',
 'Preferred_Skills',
 'Additional_Information',
 'To_Apply',
 'Hours_Shift',
 'Work_Location_1',
 'Recruitment_Contact',
 'Residency_Requirement',
 'Posting_Date',
 'Post_Until',
 'Posting_Updated',
 'Process_Date']

In [ ]:
from pyspark.sql.functions import lit, udf
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType


# Assuming 'industry_tags' is a function that takes a string and returns a string
def industry_tags_udf(industry, role, title, agency):
    return industry_tags(f"{industry}, {role}, {title}, {agency}")[0]['label']

def industry_tags_udf_score(industry, role, title, agency):
    return industry_tags(f"{industry}, {role}, {title}, {agency}")[0]['score']

# Register the UDF
industry_tags_udf = udf(industry_tags_udf, StringType())
industry_tags_udf_score = udf(industry_tags_udf_score, DoubleType())


ny_col = [ 'Agency', 'Job_Category as industry', 'Full_Time_Part_Time_indicator as job_type',
          'Salary_Range_From as min_salary', 'Salary_Range_To as max_salary',
           '__Of_Positions as vacany', 'Business_Title as role', 'Civil_Service_Title' ]

ny_df = spark.sql(f'select {",".join(ny_col)} from nyc_view where Full_Time_Part_Time_indicator = \'F\' ').collect()
ny_df = spark.createDataFrame(ny_df)

ny_df = ny_df.withColumn('description', industry_tags_udf(
    ny_df['industry'], ny_df['role'], ny_df['Civil_Service_Title'], ny_df['Agency']))

#ny_df = ny_df.withColumn('description_score', industry_tags_udf_score(ny_df['industry'], ny_df['role'], ny_df['Civil_Service_Title'], ny_df['Agency']))

ny_df = ny_df.drop('industry', 'role', 'Civil_Service_Title', 'Agency', 'job_type')

ny_df.show(5)

+----------+----------+------+--------------------+
|min_salary|max_salary|vacany|         description|
+----------+----------+------+--------------------+
|    105746|    150000|     1|Research & Consul...|
|     57490|     85912|     4|Research & Consul...|
|     60889|     78923|     1|Research & Consul...|
|     58682|    159671|     1|Construction & En...|
|     58682|    134570|     1|Construction & En...|
+----------+----------+------+--------------------+
only showing top 5 rows



In [ ]:
# bo sung chi phi thue muon cho moi noi dung tuyen dung
id_df = id_df.withColumn('min_expense', id_df['min_salary'] * id_df['vacany'])
id_df = id_df.withColumn('max_expense', id_df['max_salary'] + id_df['vacany'])
ny_df = ny_df.withColumn('min_expense', ny_df['min_salary'] * ny_df['vacany'])
ny_df = ny_df.withColumn('max_expense', ny_df['max_salary'] + ny_df['vacany'])

In [ ]:
#ny_df.write.option("header",True).csv('/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/ny_df.csv')
#id_df.write.option("header",True).csv('/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/id_df.csv')

In [ ]:
#ny_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/ny_df.csv', header=True)
#id_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/id_df.csv', header=True)

Tạo các view cho mỗi nguồn dữ liệu sau khi đã xử lý đồng bộ về cấu trúc dữ liệu


In [ ]:
ny_df.createOrReplaceTempView('gny_view')
id_df.createOrReplaceTempView('gid_view')

In [ ]:
id_df.show(5)

+----------+----------+------+--------------------+-----------+-----------+
|min_salary|max_salary|vacany|         description|min_expense|max_expense|
+----------+----------+------+--------------------+-----------+-----------+
|    6000.0|   10800.0|     1|IT Consulting & O...|     6000.0|    10801.0|
|     600.0|     600.0|     0|Construction & En...|        0.0|      600.0|
|    1800.0|    2400.0|     5|Health Care Techn...|     9000.0|     2405.0|
|    9600.0|   15600.0|     1|IT Consulting & O...|     9600.0|    15601.0|
|    3600.0|    4500.0|   200|Data Processing &...|   720000.0|     4700.0|
+----------+----------+------+--------------------+-----------+-----------+
only showing top 5 rows



In [ ]:
query = 'select description, avg(min_salary) as min_salary, avg(max_salary) as max_salary, \
 min(vacany) as vacany from '
'''
gny_df = spark.sql(f'{query} gny_view group by description').collect()
gny_df = spark.createDataFrame(gny_df)
gid_df = spark.sql(f'{query} gid_view group by description').collect()
gid_df = spark.createDataFrame(gid_df)
'''

"\ngny_df = spark.sql(f'{query} gny_view group by description').collect()\ngny_df = spark.createDataFrame(gny_df)\ngid_df = spark.sql(f'{query} gid_view group by description').collect()\ngid_df = spark.createDataFrame(gid_df)\n"

In [ ]:
#gny_df.write.option("header",True).csv('/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/gny_df.csv')
#gid_df.write.option("header",True).csv('/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/gid_df.csv')


In [ ]:
query = 'select description, avg(min_salary) as min_salary, avg(max_salary) as max_salary, \
 min(vacany) as min_vacany, max(vacany) as max_vacany from '
'''
gny_df_2 = spark.sql(f'{query} gny_view group by description').collect()
gny_df_2 = spark.createDataFrame(gny_df_2)
gid_df_2 = spark.sql(f'{query} gid_view group by description').collect()
gid_df_2 = spark.createDataFrame(gid_df_2)
'''
#gny_df_2.write.option("header",True).csv('/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/gny_df_2.csv')
#gid_df_2.write.option("header",True).csv('/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/gid_df_2.csv')


"\ngny_df_2 = spark.sql(f'{query} gny_view group by description').collect()\ngny_df_2 = spark.createDataFrame(gny_df_2)\ngid_df_2 = spark.sql(f'{query} gid_view group by description').collect()\ngid_df_2 = spark.createDataFrame(gid_df_2)\n"

In [ ]:
query = 'select description, avg(min_salary) as min_salary, avg(max_salary) as max_salary, \
 min(vacany) as min_vacany, max(vacany) as max_vacany, avg(min_expense) as min_expense, \
 avg(max_expense) as max_expense from '
gny_df_3 = spark.sql(f'{query} gny_view group by description').collect()
gny_df_3 = spark.createDataFrame(gny_df_3)
gid_df_3 = spark.sql(f'{query} gid_view group by description').collect()
gid_df_3 = spark.createDataFrame(gid_df_3)

#gny_df_3.write.option("header",True).csv('/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/gny_df_3.csv')
#gid_df_3.write.option("header",True).csv('/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/gid_df_3.csv')